In [0]:
%python


_Active User Retention_

In [0]:
create schema lemur

In [0]:

CREATE TABLE lemur.events (
    user_id INT,
    event_id INT,
    event_type STRING,
    event_date TIMESTAMP
)
USING DELTA;

-- Insert the provided data into the Delta table
INSERT INTO lemur.events VALUES
(445, 7765, 'sign-in', '2022-05-31 12:00:00'),
(445, 3634, 'like', '2022-06-05 12:00:00'),
(648, 3124, 'like', '2022-06-18 12:00:00'),
(648, 2725, 'sign-in', '2022-06-22 12:00:00'),
(648, 8568, 'comment', '2022-07-03 12:00:00'),
(445, 4363, 'sign-in', '2022-07-05 12:00:00'),
(445, 2425, 'like', '2022-07-06 12:00:00'),
(445, 2484, 'like', '2022-07-22 12:00:00'),
(648, 1423, 'sign-in', '2022-07-26 12:00:00'),
(445, 5235, 'comment', '2022-07-29 12:00:00'),
(742, 6458, 'sign-in', '2022-07-03 12:00:00'),
(742, 1374, 'comment', '2022-07-19 12:00:00');



org.apache.spark.sql.catalyst.ExtendedAnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `events` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01; line 2 pos 14;
'Project [*]
+- 'UnresolvedRelation [events], [], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.tableNotFound(package.scala:94)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$2(CheckAnalysis.scala:267)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$2$adapted(CheckAnalysis.scala:231)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:287)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:286)
	at org.ap

In [0]:
-- Query the Delta table to verify data insertion
SELECT * FROM lemur.events;

user_id,event_id,event_type,event_date
445,7765,sign-in,2022-05-31T12:00:00Z
445,3634,like,2022-06-05T12:00:00Z
648,3124,like,2022-06-18T12:00:00Z
648,2725,sign-in,2022-06-22T12:00:00Z
648,8568,comment,2022-07-03T12:00:00Z
445,4363,sign-in,2022-07-05T12:00:00Z
445,2425,like,2022-07-06T12:00:00Z
445,2484,like,2022-07-22T12:00:00Z
648,1423,sign-in,2022-07-26T12:00:00Z
445,5235,comment,2022-07-29T12:00:00Z


Assume you're given a table containing information on Facebook user actions. Write a query to obtain number of monthly active users (MAUs) in July 2022, including the month in numerical format "1, 2, 3".
An active user is defined as a user who has performed actions such as 'sign-in', 'like', or 'comment' in both the current month and the previous month.

In [0]:
with c1 as 
  (select *, date_format(event_date, 'yyyy-MM-dd'), 
  lag(event_date) over(partition by user_id order by event_date) lag
  from lemur.events)
select * from c1
where month(event_date) = 6 and month(lag) = month(event_date)-1

user_id,event_id,event_type,event_date,"date_format(event_date, yyyy-MM-dd)",lag
445,3634,like,2022-06-05T12:00:00Z,2022-06-05,2022-05-31T12:00:00Z


Y-on-Y Growth Rate

Assume you're given a table containing information about Wayfair user transactions for different products. Write a query to calculate the year-on-year growth rate for the total spend of each product, grouping the results by product ID.

The output should include the year in ascending order, product ID, current year's spend, previous year's spend and year-on-year growth percentage, rounded to 2 decimal places.

In [0]:
CREATE TABLE lemur.transactions_delta (
    transaction_id INTEGER,
    product_id INTEGER,
    spend DECIMAL(10, 2),
    transaction_date TIMESTAMP
)
USING DELTA;

-- Insert data into the Delta table
INSERT INTO lemur.transactions_delta VALUES
    (1341, 123424, 1500.60, '2019-12-31 12:00:00'),
    (1423, 123424, 1000.20, '2020-12-31 12:00:00'),
    (1623, 123424, 1246.44, '2021-12-31 12:00:00'),
    (1322, 123424, 2145.32, '2022-12-31 12:00:00'),
    (1344, 234412, 1800.00, '2019-12-31 12:00:00'),
    (1435, 234412, 1234.00, '2020-12-31 12:00:00'),
    (4325, 234412, 889.50, '2021-12-31 12:00:00'),
    (5233, 234412, 2900.00, '2022-12-31 12:00:00');

num_affected_rows,num_inserted_rows
8,8


In [0]:
with c1 as (
  select *, lag(spend) over(partition by product_id order by transaction_date) as prev
  from lemur.transactions_delta
  )
select *, round((spend-prev)/prev*100,2) diff from c1


transaction_id,product_id,spend,transaction_date,prev,diff
1341,123424,1500.60,2019-12-31T12:00:00Z,null,null
1423,123424,1000.20,2020-12-31T12:00:00Z,1500.60,-33.35
1623,123424,1246.44,2021-12-31T12:00:00Z,1000.20,24.62
1322,123424,2145.32,2022-12-31T12:00:00Z,1246.44,72.12
1344,234412,1800.00,2019-12-31T12:00:00Z,null,null
1435,234412,1234.00,2020-12-31T12:00:00Z,1800.00,-31.44
4325,234412,889.50,2021-12-31T12:00:00Z,1234.00,-27.92
5233,234412,2900.00,2022-12-31T12:00:00Z,889.50,226.03


Repeated Payments

Sometimes, payment transactions are repeated by accident; it could be due to user error, API failure or a retry error that causes a credit card to be charged twice.

Using the transactions table, identify any payments made at the same merchant with the same credit card for the same amount within 10 minutes of each other. Count such repeated payments.

Assumptions:

The first transaction of such payments should not be counted as a repeated payment. This means, if there are two transactions performed by a merchant with the same credit card and for the same amount within 10 minutes, there will only be 1 repeated payment.


In [0]:
CREATE TABLE lemur.transactions (
    transaction_id INTEGER,
    merchant_id INTEGER,
    credit_card_id INTEGER,
    transaction_timestamp TIMESTAMP,
    amount INTEGER
)
USING DELTA;

-- Insert data into the Delta table
INSERT INTO lemur.transactions VALUES
    (1, 101, 1, '2022-09-25 12:00:00', 100),
    (2, 101, 1, '2022-09-25 12:08:00', 100),
    (3, 101, 1, '2022-09-25 12:28:00', 100),
    (5, 101, 1, '2022-09-25 13:37:00', 100),
    (4, 101, 2, '2022-09-25 12:20:00', 300),
    (6, 102, 2, '2022-09-25 14:00:00', 400),
    (7, 102, 3, '2022-09-26 10:00:00', 300),
    (8, 102, 3, '2022-09-26 10:10:00', 300);

num_affected_rows,num_inserted_rows
8,8


In [0]:
select * from lemur.transactions

transaction_id,merchant_id,credit_card_id,transaction_timestamp,amount
1,101,1,2022-09-25T12:00:00Z,100
2,101,1,2022-09-25T12:08:00Z,100
3,101,1,2022-09-25T12:28:00Z,100
5,101,1,2022-09-25T13:37:00Z,100
4,101,2,2022-09-25T12:20:00Z,300
6,102,2,2022-09-25T14:00:00Z,400
7,102,3,2022-09-26T10:00:00Z,300
8,102,3,2022-09-26T10:10:00Z,300


In [0]:
with c1 as (
  select *,
  --row_number() over(partition by merchant_id, credit_card_id, amount order by transaction_timestamp asc) as row_num,
  lag(transaction_timestamp) over(partition by merchant_id, credit_card_id, amount order by transaction_timestamp asc) as lag
  from lemur.transactions
  ), c2 as (
  select *, timediff(minute, lag, transaction_timestamp) dd
  from c1)
select * from c2
where dd < 10

transaction_id,merchant_id,credit_card_id,transaction_timestamp,amount,lag,dd
2,101,1,2022-09-25T12:08:00Z,100,2022-09-25T12:00:00Z,8


In [0]:
with c1 as (
  select *,
  --row_number() over(partition by merchant_id, credit_card_id, amount order by transaction_timestamp asc) as row_num,
  lag(transaction_timestamp) over(partition by merchant_id, credit_card_id, amount order by transaction_timestamp asc) as lag
  from lemur.transactions
  ), c2 as (
  select *, timediff(minute, lag, transaction_timestamp) dd
  from c1)
select count(*) from c2
where dd < 10

count(1)
1


In [0]:
CREATE TABLE lemur.sessions (
    server_id INTEGER,
    session_status STRING,
    status_time TIMESTAMP
)
USING DELTA;

-- Insert data into the Delta table
INSERT INTO lemur.sessions VALUES
    (1, 'start', '2022-08-02 10:00:00'),
    (1, 'stop', '2022-08-04 10:00:00'),
    (1, 'stop', '2022-08-13 19:00:00'),
    (1, 'start', '2022-08-13 10:00:00'),
    (3, 'stop', '2022-08-19 10:00:00'),
    (3, 'start', '2022-08-18 10:00:00'),
    (5, 'stop', '2022-08-19 10:00:00'),
    (4, 'stop', '2022-08-19 14:00:00'),
    (4, 'start', '2022-08-16 10:00:00'),
    (3, 'stop', '2022-08-14 10:00:00'),
    (3, 'start', '2022-08-06 10:00:00'),
    (2, 'stop', '2022-08-24 10:00:00'),
    (2, 'start', '2022-08-17 10:00:00'),
    (5, 'start', '2022-08-14 21:00:00');

num_affected_rows,num_inserted_rows
14,14


In [0]:
select * from lemur.sessions

server_id,session_status,status_time
1,start,2022-08-02T10:00:00Z
1,stop,2022-08-04T10:00:00Z
1,stop,2022-08-13T19:00:00Z
1,start,2022-08-13T10:00:00Z
3,stop,2022-08-19T10:00:00Z
3,start,2022-08-18T10:00:00Z
5,stop,2022-08-19T10:00:00Z
4,stop,2022-08-19T14:00:00Z
4,start,2022-08-16T10:00:00Z
3,stop,2022-08-14T10:00:00Z


In [0]:
with c1 as 
  (select *, iff(session_status = 'start', lead(status_time) over (partition by server_id order by status_time), null) dd
  from lemur.sessions
  order by server_id, status_time),
  c2 as (
  select *, timediff(hour, status_time, dd) dd2
  from c1)
select *
from c1
where dd is not null


server_id,session_status,status_time,dd
1,start,2022-08-02T10:00:00Z,2022-08-04T10:00:00Z
1,start,2022-08-13T10:00:00Z,2022-08-13T19:00:00Z
2,start,2022-08-17T10:00:00Z,2022-08-24T10:00:00Z
3,start,2022-08-06T10:00:00Z,2022-08-14T10:00:00Z
3,start,2022-08-18T10:00:00Z,2022-08-19T10:00:00Z
4,start,2022-08-16T10:00:00Z,2022-08-19T14:00:00Z
5,start,2022-08-14T21:00:00Z,2022-08-19T10:00:00Z
